In [1]:
import fsspec
import os
import requests
import shutil

from ormhw.core import OISSTV2_DIR

## Download OISSTV2

In [2]:
def find_remote_files(years: list = range(1981, 2100), ignore_ltm: bool = True):
    """
    Find OISSTV2 daily mean sst files.
    
    :param years: A list of years to seek associated files. Default is year 1981 to 2100.
    :param ignore_ltm: Boolean to filter out long-term mean files. Default is True.
    :return: A list of urls for downloading remote files.
    """
    
    url = "https://downloads.psl.noaa.gov/Datasets/noaa.oisst.v2.highres/"
    remote = fsspec.filesystem('https')
    files = remote.glob(url + f"*sst.day.mean*")
    if ignore_ltm is True:
        files = [f for f in files if 'ltm' not in f]
    files_of_interest = []
    for year in years:
        files_of_interest += [f for f in files if str(year) in f]
    return files_of_interest


def download_files(remote_files: list, save_directory: os.path.abspath = OISSTV2_DIR, overwrite: bool = True, verbose: bool = False):
    
    """
    Download remote OISSTV2 files from a supplied list.
    
    :param remote_files: A list of urls that indicated remote file locations.
    :param save_directory: A user defined local directory for saving copies of remote files. 
    :param overwrite: Indicates whether or not to overwrite previously downloaded files. Default is True.
    :param verbose: Indicates whether to print messages. Default is False.
    :return: A list of local filepaths containing downloaded data.
    """
    
    downloaded_files = []
    os.makedirs(save_directory, exist_ok=True)
    for remote_file in remote_files:
        filename = os.path.basename(remote_file)
        filepath = os.path.join(save_directory, filename)
        if os.path.isfile(filepath):
            downloaded_files.append(filepath)
        if overwrite is False:
            if os.path.isfile(filepath):
                continue
        with requests.get(remote_file, stream = True) as req:
            with open(filepath, 'wb') as fileobj:
                shutil.copyfileobj(req.raw, fileobj)
                if os.path.isfile(filepath):
                    downloaded_files.append(filepath)
                    if verbose is True:
                        print(f'Downloaded OISSTV2: {filename}')
                else:
                    raise ConnectionError
    return downloaded_files

In [4]:
%%time
years = range(1981, 2100)
overwrite = False
remote_files = find_remote_files(years)
downloaded_files = download_files(remote_files, overwrite = overwrite)
for file in downloaded_files:
    print(file)

/home/jovyan/data/oisstv2/sst.day.mean.1981.nc
/home/jovyan/data/oisstv2/sst.day.mean.1982.nc
/home/jovyan/data/oisstv2/sst.day.mean.1983.nc
/home/jovyan/data/oisstv2/sst.day.mean.1984.nc
/home/jovyan/data/oisstv2/sst.day.mean.1985.nc
/home/jovyan/data/oisstv2/sst.day.mean.1986.nc
/home/jovyan/data/oisstv2/sst.day.mean.1987.nc
/home/jovyan/data/oisstv2/sst.day.mean.1988.nc
/home/jovyan/data/oisstv2/sst.day.mean.1989.nc
/home/jovyan/data/oisstv2/sst.day.mean.1990.nc
/home/jovyan/data/oisstv2/sst.day.mean.1991.nc
/home/jovyan/data/oisstv2/sst.day.mean.1992.nc
/home/jovyan/data/oisstv2/sst.day.mean.1993.nc
/home/jovyan/data/oisstv2/sst.day.mean.1994.nc
/home/jovyan/data/oisstv2/sst.day.mean.1995.nc
/home/jovyan/data/oisstv2/sst.day.mean.1996.nc
/home/jovyan/data/oisstv2/sst.day.mean.1997.nc
/home/jovyan/data/oisstv2/sst.day.mean.1998.nc
/home/jovyan/data/oisstv2/sst.day.mean.1999.nc
/home/jovyan/data/oisstv2/sst.day.mean.2000.nc
/home/jovyan/data/oisstv2/sst.day.mean.2001.nc
/home/jovyan/